In [ ]:
## single-nuclei Multiome analysis
## Preprocessing cellbender output files
## Andi Liu
## loading packages
library(Seurat)
library(Signac)
library(GenomicRanges)
library(EnsDb.Hsapiens.v86)
library(dplyr)
library(patchwork)
library(ggplot2)
library(future)
library(stringr)
library(tidydr)
library(factoextra)
library(DoubletFinder)
library(harmony)
library(scCustomize)

In [ ]:
## load in sequencing file metadata for each sample
sample.meta <- read.csv("Metadata.csv")
sample.meta

In [ ]:
################### ATAC annotation building ##########################
oldw <- getOption("warn")
options(warn = -1)

annotations <- GetGRangesFromEnsDb(ensdb = EnsDb.Hsapiens.v86)
seqlevelsStyle(annotations) <- 'UCSC'
genome(annotations) <- "hg38"

options(warn = oldw)

In [ ]:
########## creating a common peak sets across all independent samples ##########
# read in the atac data and convert to genomic ranges
bed.list <- list()
for (i in 1:length(sample.meta$sample_id)) {
  bed.list[[i]] <- read.table(file = paste(sample.meta[i,]$molecule_h5,"atac_peaks.bed",sep = ""),
                              col.names = c("chr", "start", "end"))
  bed.list[[i]] <- makeGRangesFromDataFrame(bed.list[[i]])
}

# Create a unified set of peaks to quantify in each dataset
combined.peaks <- reduce(x = c(bed.list[[1]],bed.list[[2]],bed.list[[3]],bed.list[[4]],
                               bed.list[[5]],bed.list[[6]],bed.list[[7]],bed.list[[8]],
                               bed.list[[9]],bed.list[[10]],bed.list[[11]],bed.list[[12]],
                               bed.list[[13]],bed.list[[14]],bed.list[[15]],bed.list[[16]],
                               bed.list[[17]],bed.list[[18]],bed.list[[19]],bed.list[[20]],
                               bed.list[[21]],bed.list[[22]]))

# Filter out bad peaks based on length
peakwidths <- width(combined.peaks)
combined.peaks <- combined.peaks[peakwidths < 10000 & peakwidths > 20] 

In [ ]:
################################################################################
##############Creating a function to QC and filter multiome data################
################################################################################
multiome_object_preprocessing <- function(folder.path,count.path,frag.path,atac.annotation, combined.peaks){
  t1 = Sys.time()
  print(t1)
  ##############################################################################
  # createing new object and conduct initial filtering -------------------------
  ##############################################################################

  # CellBender loading data
  counts <- Read_CellBender_h5_Mat(count.path)
  gex <- counts[1:36601,]
  atac <- counts[36602:dim(counts)[1],]
  print(paste("GEX raw dim",dim(gex)))
  print(paste("ATAC raw dim",dim(atac)))
     
  # extract RNA and ATAC data
  min_cells <- round(ncol(gex)/100)
  print(min_cells)
    
  # Create Seurat object
  object <- CreateSeuratObject(counts = gex,min.cells = min_cells)
    
  # cell barcode for atac seq
  cell_barcode <- colnames(object)
  
  # creating fragment object
  atac.frags <- CreateFragmentObject(
    path = frag.path,
    cells = cell_barcode
  )
  # Quantify peaks in the each dataset to create a matrix of peaks x cell using the FeatureMatrix function
  atac_counts <- FeatureMatrix(fragments = atac.frags, features = combined.peaks, 
                               cells = cell_barcode)
  print(paste("ATAC recalled dim",dim(atac_counts)))
    
  # create ATAC assay and add it to the object
  object[["ATAC"]] <- CreateChromatinAssay(
    counts = atac_counts,
    sep = c(":", "-"),
    fragments = atac.frags,
    annotation = atac.annotation,
    genome = "hg38",
    min.cells = 10
  )
  
  DefaultAssay(object) <- "ATAC"
  object <- NucleosomeSignal(object)
  object <- TSSEnrichment(object)
  
  # filtering based on parameters
  object <- subset(
    x = object,
    subset = nFeature_RNA < 10000 &
      nFeature_RNA > 200 &
      nucleosome_signal < 2.5 &
      TSS.enrichment >2
  )
  ##############################################################################
  # calculate k means n = 2 based on mitocondril reads -------------------------
  ##############################################################################
  DefaultAssay(object) <- "RNA"
  # calculating MT reads
  object[["percent.mt"]] <- PercentageFeatureSet(object, pattern = "^MT-")
  
  # then keep the group with lower mean valus 
  mt_kmeans <- kmeans(object$percent.mt,centers = 2)
  print(mt_kmeans$centers)
  print(mt_kmeans$size)
  
  object$mt_keep <- mt_kmeans$cluster 
  if(mt_kmeans$centers[1] > mt_kmeans$centers[2]){
    object <- subset(x = object,subset = mt_keep == 2)
  }else{
    object <- subset(x = object,subset = mt_keep == 1)
  }
  
  ##############################################################################
  # identify doublet cells based on gene expression profile
  # assuming a 5% doublet per dataset
  ##############################################################################
  ## Pre-process Seurat object (Log normalize) ----------------------------------------------------------------------------------- # nolint
  object <- NormalizeData(object)
  object <- FindVariableFeatures(object, selection.method = "vst", nfeatures = 2000)
  object <- ScaleData(object)
  object <- RunPCA(object,verbose = F)
  object <- RunUMAP(object, dims = 1:10,verbose = F)
  
  ## pK Identification (no ground-truth) ---------------------------------------------------------------------------------------
  sweep.res.list <- paramSweep(object, PCs = 1:10, sct = F)
  sweep.stats <- summarizeSweep(sweep.res.list, GT = FALSE)
  bcmvn <- find.pK(sweep.stats)
  ## Homotypic Doublet Proportion Estimate -------------------------------------------------------------------------------------
  nExp_poi <- round(0.05*nrow(object@meta.data))  ## Assuming 5% doublet formation rate - tailor for your dataset
  ## Run DoubletFinder with varying classification stringencies ----------------------------------------------------------------
  pK_optimal <- as.numeric(as.character(bcmvn[which.max(bcmvn$BCmetric),]$pK))
  object <- doubletFinder(object, PCs = 1:10, pN = 0.25, pK = pK_optimal, nExp = nExp_poi, reuse.pANN = FALSE, sct = FALSE)
  
  return(object) 
  
  # get time
  t2 = Sys.time()
  print(t2-t1)
}

In [ ]:
################################################################################
######################## create the multiome object(s) #########################
################################################################################
#creating a empty object list
object.list <- list()
i = 1
# read in data and preprocessing the data based on the parameters
for (i in 1:length(sample.meta$sample_id)) {
  print(paste("Sample",sample.meta[i,]$sample_id,"on processing."))
  # generating two paths
  folder.path <- sample.meta[i,]$molecule_h5
  count.path <- sample.meta$cellbender_output[i]
  frag.path <-  paste(sample.meta[i,]$molecule_h5,"atac_fragments.tsv.gz",sep = "")
    
  # preproceesing each multiome object
  object.list[[i]] <- multiome_object_preprocessing(
    folder.path = folder.path,
    count.path = count.path,
    frag.path = frag.path,
    atac.annotation = annotations,
    combined.peaks = combined.peaks
  )
  print(paste("Sample",sample.meta[i,]$sample_id,"fisnished!"))
}
print(object.list)
# better to clean output after running..the output takes a lot of space.

In [ ]:
################################################################################
############ Doublet removal based on DoubletFinder results ####################
################################################################################
for (i in 1:length(object.list)){
    #print(i)
    meta <- object.list[[i]]@meta.data
    meta$doublet <- ifelse(meta[,13] == "Doublet",T,F)
    # deleting the two columns contain redundant information
    meta <- meta[,-c(12:13)]
    object.list[[i]]@meta.data <- meta
    
    object.list[[i]] <- subset(object.list[[i]],subset = doublet == F)
}

#object.list

In [ ]:
################################################################################
### Merging all individual object to a merged object for downstream analysis ###
################################################################################

## removing data and scale.data because of merging time. 
for (i in 1:length(object.list)){
  DefaultAssay(object.list[[i]]) <- "RNA"
  object.list[[i]][["RNA"]]$data <- NULL
  object.list[[i]][["RNA"]]$scale.data <- NULL
}

# merging and adding sample id
object.merge <- merge(object.list[[1]],y = c(object.list[[2]],object.list[[3]],
                                             object.list[[4]],object.list[[5]],
                                             object.list[[6]],object.list[[7]],
                                             object.list[[8]],object.list[[9]],
                                            object.list[[10]],object.list[[11]],
                                            object.list[[12]],object.list[[13]],
                                            object.list[[14]],object.list[[15]],
                                            object.list[[16]],object.list[[17]],
                                            object.list[[18]],object.list[[19]],
                                            object.list[[20]],object.list[[21]],
                                            object.list[[22]]),
                      add.cell.ids = sample.meta$sample_id)

# individual ID
meta <- object.merge@meta.data
meta$individual_ID <-  str_split_fixed(rownames(meta),"_",3)[,1]
# batch information
meta$batch <- NA
meta[meta$individual_ID %in% c("UT04","UT09","UT2105","UT2106"),]$batch <- 2
meta[meta$individual_ID %in% c("NIH04","NIH10","NIH13","NIH16"),]$batch <- 3
meta[meta$individual_ID %in% c("NIH01","NIH02","NIH03","NIH17","NIH18","NIH28","NIH29","NIH30"),]$batch <- 4
meta[meta$individual_ID %in% c("NIH05","NIH06","NIH11","NIH12","NIH14","NIH15"),]$batch <- 5

# diagnosis information
meta$diagnosis <- ifelse(
  meta$individual_ID %in% c("UT2105","UT2106",
                            "NIH10","NIH11","NIH12",
                            "NIH13","NIH14","NIH15",
                            "NIH16","NIH17","NIH18"),"EOAD","NCI")

# brain region information
meta$regions <- NA
meta[meta$individual_ID %in% c("UT04","UT09","UT2105","UT2106",
                              "NIH01","NIH04","NIH07","NIH10",
                              "NIH13","NIH16","NIH28"),]$regions <- "PFC"
meta[meta$individual_ID %in% c("NIH02","NIH05","NIH08","NIH11",
                              "NIH14","NIH17","NIH29"),]$regions <- "EC"
meta[meta$individual_ID %in% c("NIH03","NIH06","NIH12","NIH15",
                              "NIH18","NIH30"),]$regions <- "HIP"

# assign back the meta data
object.merge@meta.data <- meta

## checking and saving
ls()
print(object.merge)

In [ ]:
### Since using seurat V5, need to combine(Join) layers of data from each individual. Taking long time....
DefaultAssay(object.merge) <- "RNA"
# removing unused data might speed up the procedures. for example: object[['RNA']]$scale.data.1 <- NULL
object.merge[["RNA"]] <- JoinLayers(object.merge[["RNA"]]) # must add [["XXX"]], otherwise takes forever.....
## checking joined dataset about the dimension, etc. 
print(object.merge)
Layers(object.merge[["RNA"]]) # double check layers
#save(object.list,annotations,combined.peaks,object.merge, file = "01.cellbender_merged_object_1.3.RData")

In [ ]:
### checking dimension ###
## since using seurat V5 check dim
meta <- object.merge@meta.data
table(meta$individual_ID)

## check ATAC fragments file paths
Fragments(object.merge[["ATAC"]])[[20]]@path

In [ ]:
######################################################################
#### create new layer of data containing only protein coding genes ###
######################################################################
# loading
PCgene <- read.delim("./Data/protein-coding_gene.txt")

# creating
DefaultAssay(object.merge) <- "RNA"
table(rownames(object.merge) %in% PCgene$symbol)
temp.assay <- subset(object.merge[["RNA"]], features = PCgene$symbol)
Key(temp.assay) <- "pc_"
object.merge[["PC"]] <- temp.assay
# clean temp file
rm(temp.assay)

In [ ]:
######################################################################
################ first normalization and clustering ##################  
######################################################################
# protein coding layer
DefaultAssay(object.merge) <- "PC"

object.merge <- SCTransform(object.merge,assay = "PC",new.assay.name = "SCT",vars.to.regress = c("percent.mt"),layer = "counts",verbose = T,method = "glmGamPoi") %>% 
        RunPCA(ndims=30) %>% 
        FindNeighbors(dims = 1:30) %>% 
        RunUMAP(dims = 1:30, reduction.name="umap.rna",reduction.key = "rnaUMAP_")

# ATAC layer (common peak set)
DefaultAssay(object.merge)<-"ATAC"
# Normalization and UMAP on ATAC seq 
object.merge <- RunTFIDF(object.merge) %>% 
                FindTopFeatures(min.cutoff='q0') %>% 
                RunSVD() %>% 
                RunUMAP(reduction='lsi',dims=2:50,reduction.name="umap.atac",reduction.key="atacUMAP_") 

object.merge <- FindMultiModalNeighbors(object.merge, reduction.list=list("pca","lsi"), dims.list=list(1:30,2:50))
object.merge <- RunUMAP(object.merge,nn.name="weighted.nn", reduction.name="wnn.umap",reduction.key="wnnUMAP_")
object.merge <- FindClusters(object.merge, graph.name="wsnn", algorithm=3)

In [ ]:
######################################################################
###########-- FIRST UMAP of unintegrated overall raw data--###########
######################################################################
umap_theme <- theme_dr()+theme(panel.grid.major = element_blank(), 
                                            panel.grid.minor = element_blank(),
                                            panel.background = element_blank(), 
                                            axis.line = element_line(colour = "black"))

p1 <- DimPlot(object.merge, label = T, repel = TRUE, 
              reduction = "umap.rna",group.by = "individual_ID")+umap_theme+ggtitle("RNA")
p2 <- DimPlot(object.merge, label = T, repel = TRUE, 
              reduction = "umap.atac",group.by = "individual_ID")+umap_theme+ggtitle("peaks")
p3 <- DimPlot(object.merge, label = T, repel = TRUE, 
              reduction = "wnn.umap",group.by = "individual_ID")+umap_theme+ggtitle("WNN")

p4 <- DimPlot(object.merge, label = T, repel = TRUE, 
              reduction = "wnn.umap",group.by = "batch")+umap_theme+ggtitle("batch_WNN")
p5 <- DimPlot(object.merge, label = T, repel = TRUE, 
              reduction = "wnn.umap",group.by = "diagnosis")+umap_theme+ggtitle("diagnosis_WNN")
p6 <- DimPlot(object.merge, label = T, repel = TRUE, 
              reduction = "wnn.umap",group.by = "seurat_clusters")+umap_theme+ggtitle("cluster_WNN")

ggsave(filename = "./Figures/UMAP/0_merged.object_UMAP_individual.pdf",
    patchwork::wrap_plots(p1,p2,p3, ncol = 3),
      scale = 1, width = 17, height = 5)

ggsave(filename = "./Figures/UMAP/0_merged.object_UMAP_batche_and_diagnosis.pdf",
    patchwork::wrap_plots(p4,p5,p6, ncol = 3),
      scale = 1, width = 17, height = 5)

In [ ]:
################################################################################
#################### Harmony integration on each omic ##########################
################################################################################
library(harmony)

## Normalization and clutering
DefaultAssay(object.merge) <- "PC"

## RNA harmony integration
integrated<-RunHarmony(object.merge, group.by.vars="individual_ID",reduction="pca",assay.use="SCT", 
                       reduction.save="harmony.rna",theta = 1)
integrated<-RunUMAP(integrated,reduction="harmony.rna",dims=1:30,reduction.name="harmony.rna.umap")

# finding RNA clusters
integrated<-FindNeighbors(integrated,dims=1:30, reduction="harmony.rna")
integrated<-FindClusters(integrated, graph.name="SCT_nn", resolution=c(0.8))
integrated$RNA_harmony_cluster <- integrated$seurat_clusters

## ATAC part (no call peaks yet)
DefaultAssay(integrated)<-"ATAC"

## ATAC harmony integration
integrated <- RunHarmony(integrated, group.by.vars="individual_ID", reduction="lsi",
                       assay.use="ATAC",reduction.save="harmony.atac", project.dim=F)
integrated <- RunUMAP(integrated,reduction="harmony.atac",dims=1:30,reduction.name="harmony.atac.umap")

## integration and jointly clustering
integrated <- FindMultiModalNeighbors(integrated, reduction.list=list("harmony.rna","harmony.atac"), dims.list=list(1:30,2:50))
integrated <- RunUMAP(integrated,nn.name="weighted.nn", reduction.name="wnn.umap",reduction.key="wnnUMAP_")
integrated <- FindClusters(integrated, graph.name = "wsnn", algorithm = 3, verbose = T, resolution = c(0.8))
integrated$WNN_harmony_cluster <- integrated$seurat_clusters

In [ ]:
######################################################################
#########-- 02_second UMAP of integrated overall  data  --############
######################################################################
umap_theme <- theme_dr()+theme(panel.grid.major = element_blank(), 
                                            panel.grid.minor = element_blank(),
                                            panel.background = element_blank(), 
                                            axis.line = element_line(colour = "black"))
## transition from unintegrated to harmony integration
p1 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "umap.rna",group.by = "individual_ID")+umap_theme+ggtitle("unintegrated_RNA")
p2 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "umap.atac",group.by = "individual_ID")+umap_theme+ggtitle("unintegrated_combined_peaks")

p3 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "harmony.rna.umap",group.by = "individual_ID")+umap_theme+ggtitle("harmony_RNA")
p4 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "harmony.atac.umap",group.by = "individual_ID")+umap_theme+ggtitle("harmony_combined_peaks")

ggsave(filename = "./Figures/UMAP/1_Harmony_integration_GEX_ATAC.pdf",
    patchwork::wrap_plots(p1,p2,p3,p4, ncol = 2),
      scale = 1, width = 12, height = 10)

## joint harmony embedding
p5 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "wnn.umap",group.by = "batch")+umap_theme+ggtitle("harmony_WNN_batch")
p6 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "wnn.umap",group.by = "diagnosis")+umap_theme+ggtitle("harmony_WNN_diagnosis")
p7 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "wnn.umap",group.by = "individual_ID")+umap_theme+ggtitle("harmony_WNN")
p8 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "wnn.umap",group.by = "WNN_harmony_cluster")+umap_theme+ggtitle("harmony_WNN_clusters")

ggsave(filename = "./Figures/UMAP/1_Harmony_integration_UMAP_clusters.pdf",
    patchwork::wrap_plots(p5,p6,p7,p8, ncol = 2),
      scale = 1, width = 12, height = 10)

In [ ]:
########################################################################
## first normalization and clustering for ploting the gene expression ##
########################################################################
# protein coding layer
DefaultAssay(integrated) <- "PC"

integrated <- NormalizeData(integrated)
integrated <- FindVariableFeatures(integrated, selection.method = "vst", nfeatures = 3000)
integrated <- ScaleData(integrated) %>%
        RunPCA(ndims=30) %>%
        FindNeighbors(dims = 1:30)

print(integrated)

In [ ]:
#####################################################
#########-- saving file and reloading  --############
#####################################################
save(integrated, file = "01.integrated_multiome_object.RData")

In [ ]:
## based on pseudobulk analysis results, kick out outliers
integrated <- subset(integrated,subset = individual_ID != "UT2106")
meta <- integrated@meta.data
length(meta$individual_ID)
table(meta$individual_ID)

In [ ]:
################################################################################
########## redo normalization, integration for visualization ###################
################################################################################
library(harmony)

## Normalization and clutering
DefaultAssay(integrated) <- "RNA"

integrated <- SCTransform(integrated,assay="RNA",new.assay.name="SCT",vars.to.regress = c("percent.mt"),verbose=T) %>% 
        RunPCA(ndims=30) %>% 
        FindNeighbors(dims = 1:30) %>% 
        RunUMAP(dims = 1:30, reduction.name="umap.rna",reduction.key = "rnaUMAP_")

## RNA harmony integration
integrated<-RunHarmony(integrated, group.by.vars="individual_ID",reduction="pca",assay.use="SCT", 
                       reduction.save="harmony.rna",theta = 1)
integrated<-RunUMAP(integrated,reduction="harmony.rna",dims=1:30,reduction.name="harmony.rna.umap")

# finding RNA clusters
integrated<-FindNeighbors(integrated,dims=1:30, reduction="harmony.rna")
integrated<-FindClusters(integrated, graph.name="SCT_nn", resolution=c(0.8))
integrated$RNA_harmony_cluster <- integrated$seurat_clusters

## ATAC part (no call peaks yet)
DefaultAssay(integrated)<-"ATAC"

# Normalization and UMAP on ATAC seq 
integrated <- RunTFIDF(integrated) %>% 
                FindTopFeatures(min.cutoff='q0') %>% 
                RunSVD() %>% 
                RunUMAP(reduction='lsi',dims=2:50,reduction.name="umap.atac",reduction.key="atacUMAP_") 

## ATAC harmony integration
integrated <- RunHarmony(integrated, group.by.vars="individual_ID", reduction="lsi",
                       assay.use="ATAC",reduction.save="harmony.atac", project.dim=F)
integrated <- RunUMAP(integrated,reduction="harmony.atac",dims=1:30,reduction.name="harmony.atac.umap")

## integration and jointly clustering
integrated <- FindMultiModalNeighbors(integrated, reduction.list=list("harmony.rna","harmony.atac"), dims.list=list(1:30,2:50))
integrated <- RunUMAP(integrated,nn.name="weighted.nn", reduction.name="wnn.umap",reduction.key="wnnUMAP_")
integrated <- FindClusters(integrated, graph.name = "wsnn", algorithm = 3, verbose = T, resolution = c(0.8))

In [ ]:
######################################################################
#########-- 02_second UMAP of integrated overall  data  --############
######################################################################
umap_theme <- theme_dr()+theme(panel.grid.major = element_blank(), 
                                            panel.grid.minor = element_blank(),
                                            panel.background = element_blank(), 
                                            axis.line = element_line(colour = "black"))
## transition from unintegrated to harmony integration
p1 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "harmony.rna.umap",group.by = "individual_ID")+umap_theme+ggtitle("harmony.rna.umap.rmOutlier")
p2 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "harmony.atac.umap",group.by = "individual_ID")+umap_theme+ggtitle("harmony.atac.umap.rmOutlier")
p3 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "wnn.umap",group.by = "individual_ID")+umap_theme+ggtitle("Joint_harmony_common_peaks_WNN_individual_rmOutliers")
p4 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "wnn.umap",group.by = "seurat_clusters")+umap_theme+ggtitle("joint_harmony_common_peaks_WNN_clusters.rmOutliers")

ggsave(filename = "./Figures/UMAP/2_harmony_integrated_object_rmOurliers.pdf",
    patchwork::wrap_plots(p1,p2,p3,p4, ncol = 2),
      scale = 1, width = 12, height = 10)
#patchwork::wrap_plots(p1,p2,p3,p4, ncol = 2)

In [ ]:
########################################################################
## first normalization and clustering for ploting the gene expression ##
########################################################################
# protein coding layer
DefaultAssay(integrated) <- "PC"

integrated <- NormalizeData(integrated)
integrated <- FindVariableFeatures(integrated, selection.method = "vst", nfeatures = 3000)
integrated <- ScaleData(integrated) %>%
        RunPCA(ndims=30) %>%
        FindNeighbors(dims = 1:30)

print(integrated)

In [ ]:
######################################################################
###########-- check cell type marker genes on each cluster --#########
######################################################################
DefaultAssay(integrated) <- "PC"
p3 <- FeaturePlot(integrated,features = c("SYT1",#neuron in general
                                          "NRGN","CAMK2A",# excitatory neurons
                                          "GAD1","GAD2","SST", # inhibitory neurons
                                          "AQP4","GFAP", # astrocytes
                                          "CD74","CSF1R","C3",# microglia
                                          "MOBP","MBP",# oligodendrocyte
                                          "VCAN","PDGFRA", # OPCs
                                          "FLT1",# endotheial
                                          "HIGD1B" # VLMC/Pericyte
                                         ), ncol = 8,slot = "data",
                  reduction = "harmony.rna.umap")
ggsave(filename = "./Figures/UMAP/2_CT_features.png",
    patchwork::wrap_plots(p3, ncol = 1),
      scale = 1, width = 40, height = 15)

In [ ]:
######################################################################
## Perform cell type annotation based on file from SEA-AD reference ##
######################################################################
## loading reference datasets
load("./Reference/02_reference_SEA_AD_1234_subclass.RData")
load("./Reference//01_Reference_ROSMAP_Control.RData")
ls()

meta_ref_seaad <- reference_1234_subclass@meta.data
meta_ref_methy <- reference@meta.data

## change name
meta_ref_methy$major.cell.type <- NA
meta_ref_methy[meta_ref_methy$broad.cell.type == "Ex",]$major.cell.type <- "Excitatory"
meta_ref_methy[meta_ref_methy$broad.cell.type == "In",]$major.cell.type <- "Inhibitory"
meta_ref_methy[meta_ref_methy$broad.cell.type == "Ast",]$major.cell.type <- "Astrocyte"
meta_ref_methy[meta_ref_methy$broad.cell.type == "End",]$major.cell.type <- "Endothelial"
meta_ref_methy[meta_ref_methy$broad.cell.type == "Mic",]$major.cell.type <- "Microglia"
meta_ref_methy[meta_ref_methy$broad.cell.type == "Oli",]$major.cell.type <- "Oligodendrocyte"
meta_ref_methy[meta_ref_methy$broad.cell.type == "Opc",]$major.cell.type <- "OPC"
meta_ref_methy[meta_ref_methy$broad.cell.type == "Per",]$major.cell.type <- "VLMC/Per"

# for seaad
meta_ref_seaad[meta_ref_seaad$major.cell.type == "Microglia-PVM",]$major.cell.type <- "Microglia"
meta_ref_seaad[meta_ref_seaad$major.cell.type == "VLMC",]$major.cell.type <- "VLMC/Per"

table(meta_ref_seaad$major.cell.type)
table(meta_ref_methy$major.cell.type)

reference_1234_subclass@meta.data <- meta_ref_seaad
reference@meta.data <- meta_ref_methy

In [ ]:
#### annotation from seaad ####
DefaultAssay(integrated) <- "SCT"
integrated$barcode <- str_split_fixed(rownames(integrated@meta.data), 
                                   pattern = "_",n = 2)[,2]

# transfer cell type labels from reference to query
transfer_anchors <- FindTransferAnchors(
  reference = reference_1234_subclass,
  query = integrated,
  normalization.method = "SCT",
  query.assay = "SCT",
  recompute.residuals = FALSE,
  dims = 1:30
)

## transferring the major cell type label
Idents(reference_1234_subclass) <- "major.cell.type"
predictions <- TransferData(
  anchorset = transfer_anchors, 
  reference = reference_1234_subclass,
  refdata = reference_1234_subclass$major.cell.type,#sublass_label
  query = integrated,
  dims = 1:30
)

integrated$major.cell.type.prediction.seaad <- predictions$predicted.id
integrated$major.cell.type.prediction.seaad.score <- predictions$predicted.id.score

In [ ]:
#### annotation from Mathy et al. ####
DefaultAssay(integrated) <- "SCT"
integrated$barcode <- str_split_fixed(rownames(integrated@meta.data), 
                                   pattern = "_",n = 2)[,2]

# transfer cell type labels from reference to query
transfer_anchors <- FindTransferAnchors(
  reference = reference,
  query = integrated,
  normalization.method = "SCT",
  query.assay = "SCT",
  recompute.residuals = FALSE,
  dims = 1:30
)

## transferring the major cell type label
Idents(reference) <- "major.cell.type"
predictions <- TransferData(
  anchorset = transfer_anchors, 
  reference = reference,
  refdata = reference$major.cell.type,#sublass_label
  query = integrated,
  dims = 1:30
)

integrated$major.cell.type.prediction.mathy <- predictions$predicted.id
integrated$major.cell.type.prediction.mathy.score <- predictions$predicted.id.score

In [ ]:
## check overlap between two reference
meta <- integrated@meta.data
table(meta$major.cell.type.prediction.seaad)
table(meta$major.cell.type.prediction.mathy)
table(meta$major.cell.type.prediction.seaad == meta$major.cell.type.prediction.mathy) # check consistency

# check how many cells will be removed# check consistency plus confidency
integrated$kick <- ifelse(
    meta$major.cell.type.prediction.seaad == meta$major.cell.type.prediction.mathy & 
    meta$major.cell.type.prediction.seaad.score >= 0.95 & 
    meta$major.cell.type.prediction.mathy.score >= 0.95,
    T,F)

# check number
table(integrated$kick)

In [ ]:
umap_theme <- theme_dr()+theme(panel.grid.major = element_blank(), 
                                            panel.grid.minor = element_blank(),
                                            panel.background = element_blank(), 
                                            axis.line = element_line(colour = "black"))
p1 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "harmony.rna.umap",group.by = "RNA_harmony_cluster")+umap_theme+ggtitle("seurat_clusters")
p2 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "harmony.rna.umap",group.by = "major.cell.type.prediction.seaad")+umap_theme+ggtitle("CT_by_SEA-AD")
p3 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "harmony.rna.umap",group.by = "major.cell.type.prediction.mathy")+umap_theme+ggtitle("CT_by_Mathy")
p4 <- DimPlot(integrated, label = T, repel = TRUE, 
              reduction = "harmony.rna.umap",group.by = "kick")+umap_theme+ggtitle("low predicability")
ggsave(filename = "./Figures/UMAP/2_integrated_object_CT_prediction.pdf",
    patchwork::wrap_plots(p1,p2,p3,p4, ncol = 2),
      scale = 1, width = 12, height = 10)

In [ ]:
#################################################################
##-- preparing for scANVI model for sub-cellclass annotation --##
#################################################################
##conda install numpy scipy joblib scikit-learn --force-reinstall
## since using seurat 5, need to change some assay to seurat 3/4 for some functions
# backup
outobj <- integrated
outobj[["RNA"]] <- as(outobj[["RNA"]], "Assay")
DefaultAssay(outobj) <- "RNA"
outobj
## output as a h5ad format, need to install the following things

DefaultAssay(outobj) <- "RNA"
#integrated@assays$RNA$data <- NULL
sceasy::convertFormat(outobj, from="seurat", to="anndata",
                       outFile='integrated_object_for_scvi_raw.h5ad')

In [ ]:
#################################################################
##-- kick the cells with low cell type annotation confidency --##
#################################################################
## subsetting as the clean dataset
clean <- subset(integrated, subset = kick ==T)
clean

In [ ]:
################################################################################
########## redo normalization, integration for visualization ###################
################################################################################
## Normalization and clutering
## RNA harmony integration
clean<-RunHarmony(clean, group.by.vars="individual_ID",reduction="pca",assay.use="SCT", 
                       reduction.save="harmony.rna",theta = 1)
clean<-RunUMAP(clean,reduction="harmony.rna",dims=1:30,reduction.name="harmony.rna.umap")
# finding RNA clusters
clean<-FindNeighbors(clean,dims=1:30, reduction="harmony.rna")
clean<-FindClusters(clean, graph.name="SCT_nn", resolution=2.8)
clean$RNA_harmony_cluster <- integrated$seurat_clusters

In [ ]:
## remove the cells with different label with majority of the clusters for cleaning 
#filter out cells that are not assigned the same cell type as their neighbors. Additional doublet precaution 
Idents(clean)<-clean$RNA_harmony_cluster
tab<-table(clean$RNA_harmony_cluster, clean$major.cell.type.prediction.mathy)
tmp<-colnames(tab)[apply(tab,1,which.max)]
names(tmp)<-levels(clean)
clean<-RenameIdents(clean,tmp)
clean$cluster_celltype<-Idents(clean)
clean$agree<-ifelse(clean$cluster_celltype==clean$major.cell.type.prediction.mathy, 1,0)

table(clean$agree)

In [ ]:
clean <- subset(clean, subset=agree==1) 
clean

In [ ]:
################################################################################
############### redo normalization, integration for visualization ##############
#################### Harmony integration on each omic ##########################
################################################################################
library(harmony)

## Normalization and clutering
DefaultAssay(clean) <- "PC"

clean <- SCTransform(clean,assay="PC",new.assay.name="SCT",vars.to.regress = c("percent.mt"),verbose=T) %>% 
        RunPCA(ndims=30) %>% 
        FindNeighbors(dims = 1:30) %>% 
        RunUMAP(dims = 1:30, reduction.name="umap.rna",reduction.key = "rnaUMAP_")

## RNA harmony integration
clean<-RunHarmony(clean, group.by.vars="individual_ID",reduction="pca",assay.use="SCT", 
                       reduction.save="harmony.rna",theta = 1)
clean<-RunUMAP(clean,reduction="harmony.rna",dims=1:30,reduction.name="harmony.rna.umap")

# finding RNA clusters
clean<-FindNeighbors(clean,dims=1:30, reduction="harmony.rna")
clean<-FindClusters(clean, graph.name="SCT_nn", resolution=1)
clean$RNA_harmony_cluster <- integrated$seurat_clusters

## ATAC part (no call peaks yet)
DefaultAssay(clean)<-"ATAC"

# Normalization and UMAP on ATAC seq 
clean <- RunTFIDF(clean) %>% 
                FindTopFeatures(min.cutoff='q0') %>% 
                RunSVD() %>% 
                RunUMAP(reduction='lsi',dims=2:50,reduction.name="umap.atac",reduction.key="atacUMAP_") 

## ATAC harmony integration
clean <- RunHarmony(clean, group.by.vars="individual_ID", reduction="lsi",
                       assay.use="ATAC",reduction.save="harmony.atac", project.dim=F)
clean <- RunUMAP(clean,reduction="harmony.atac",dims=1:30,reduction.name="harmony.atac.umap")

## integration and jointly clustering
clean <- FindMultiModalNeighbors(clean, reduction.list=list("harmony.rna","harmony.atac"), dims.list=list(1:30,2:50))
clean <- RunUMAP(clean,nn.name="weighted.nn", reduction.name="wnn.umap",reduction.key="wnnUMAP_")
clean <- FindClusters(clean, graph.name = "wsnn", algorithm = 3, verbose = T, resolution = 1)

In [ ]:
umap_theme <- theme_dr()+theme(panel.grid.major = element_blank(), 
                                            panel.grid.minor = element_blank(),
                                            panel.background = element_blank(), 
                                            axis.line = element_line(colour = "black"))
p1 <- DimPlot(clean, label = T, repel = TRUE, 
              reduction = "harmony.rna.umap",group.by = "seurat_clusters")+umap_theme+ggtitle("seurat_clusters")
p2 <- DimPlot(clean, label = T, repel = TRUE, 
              reduction = "harmony.rna.umap",group.by = "major.cell.type.prediction.seaad")+umap_theme+ggtitle("CT_by_SEA-AD")
p3 <- DimPlot(clean, label = T, repel = TRUE, 
              reduction = "harmony.rna.umap",group.by = "major.cell.type.prediction.mathy")+umap_theme+ggtitle("CT_by_Mathy")

ggsave(filename = "./Figures/UMAP/3_clean_object_CT.pdf",
    patchwork::wrap_plots(p1,p2,p3, ncol = 3),
      scale = 1, width = 17, height = 5)

In [ ]:
## saving
saveRDS(clean,file = "03.clean_object.10.31.rds")

In [ ]:
library(raster)
library(GenomicRanges)

In [ ]:
library(EnsDb.Hsapiens.v86)
#redo peak calling using MACS2 for the combined Seurat object, with the group.by parameter set to the 
#orig.ident (sample ID) to identify peaks for each sample separately
combined_peaks_macs2 <- CallPeaks(clean, assay = "ATAC", group.by = "cluster_celltype",
        outdir = "./Analysis/tempout/",
        fragment.tempdir = "./Analysis/tempout/",
        combine.peaks = TRUE)

In [ ]:
df <- as.data.frame(combined_peaks_macs2)
write.table(df[,1:3], file="./Results/CTpeak_set_3.28.bed", quote=F, sep="\t", row.names=F, col.names=F)
write.csv(df,"./Analysis/Results/CTpeaks_annotated.csv")

In [ ]:
# quantify counts in each peak
macs2_counts_combined <- FeatureMatrix(fragments = clean@assays$ATAC@fragments, 
                                       features = combined_peaks_macs2, cells = colnames(clean))

clean[["CTpeaks"]] <- CreateChromatinAssay(counts = macs2_counts_combined, 
                                           fragments = clean@assays$ATAC@fragments, 
                                           annotation = clean@assays$ATAC@annotation, 
                                           genome = "hg38")

In [ ]:
library(EnsDb.Hsapiens.v86)
# try call peaks again and examine the character
DefaultAssay(clean) <- "ATAC"

# call peaks using MACS2
peaks <- CallPeaks(clean,
        fragment.tempdir = "./Analysis/tempout/",
        combine.peaks = TRUE)

# call peaks using MACS2
peaks <- CallPeaks(clean)

# remove peaks on nonstandard chromosomes and in genomic blacklist regions
peaks <- keepStandardChromosomes(peaks, pruning.mode = "coarse")
peaks <- subsetByOverlaps(x = peaks, ranges = blacklist_hg38_unified, invert = TRUE)

df <- as.data.frame(peaks)
#bed file for reanalyze [sort -k1,1 -k2,2n CT_peak_set.bed]
write.table(df[,1:3], file="./Results/macs2_peaks_set.bed", quote=F, sep="\t", row.names=F, col.names=F)
write.csv(df,"./Results/macs2_peaks_annotated.csv")

# quantify counts in each peak
macs2_counts_combined <- FeatureMatrix(fragments = clean@assays$ATAC@fragments, 
                                       features = peaks, cells = colnames(clean))

clean[["peaks"]] <- CreateChromatinAssay(counts = macs2_counts_combined, 
                                           fragments = clean@assays$ATAC@fragments, 
                                           annotation = clean@assays$ATAC@annotation, 
                                           genome = "hg38")

In [ ]:
##-- reload and final preprocess before downstream analysis --##
#save(clean,file = "03.clean_object_rmOutlier.1.10.RData")
#load("03.clean_object_rmOutlier.RData")
ls()
clean <- readRDS("./03.clean_object.10.31.rds")
clean

In [ ]:
## redo normalization, integration for visualization
################################################################################
#################### Harmony integration on each omic ##########################
################################################################################
library(harmony)

## Normalization and clutering
DefaultAssay(clean) <- "PC"

clean <- SCTransform(clean,assay="PC",new.assay.name="SCT",vars.to.regress = c("percent.mt"),verbose=T) %>% 
        RunPCA(ndims=30) %>% 
        FindNeighbors(dims = 1:30) %>% 
        RunUMAP(dims = 1:30, reduction.name="umap.rna",reduction.key = "rnaUMAP_")

## RNA harmony integration
clean<-RunHarmony(clean, group.by.vars="individual_ID",reduction="pca",assay.use="SCT", 
                       reduction.save="harmony.rna",theta = 1)
clean<-RunUMAP(clean,reduction="harmony.rna",dims=1:30,reduction.name="harmony.rna.umap")

# finding RNA clusters
clean<-FindNeighbors(clean,dims=1:30, reduction="harmony.rna")
clean<-FindClusters(clean, graph.name="SCT_nn", resolution=1)
clean$RNA_harmony_cluster <- clean$seurat_clusters

## CTpeaks part (no call peaks yet)
DefaultAssay(clean)<-"CTpeaks"

# Normalization and UMAP on ATAC seq 
clean <- RunTFIDF(clean) %>% 
                FindTopFeatures(min.cutoff='q0') %>% 
                RunSVD() %>% 
                RunUMAP(reduction='lsi',dims=2:50,reduction.name="umap.CTpeaks",reduction.key="CTpeaksUMAP_") 

## ATAC harmony integration
clean <- RunHarmony(clean, group.by.vars="individual_ID", reduction="lsi",
                       assay.use="CTpeaks",reduction.save="harmony.CTpeaks", project.dim=F)
clean <- RunUMAP(clean,reduction="harmony.CTpeaks",dims=1:30,reduction.name="harmony.CTpeaks.umap")


## peaks part (no call peaks yet)
DefaultAssay(clean)<-"peaks"

# Normalization and UMAP on ATAC seq 
clean <- RunTFIDF(clean) %>% 
                FindTopFeatures(min.cutoff='q0') %>% 
                RunSVD() %>% 
                RunUMAP(reduction='lsi',dims=2:50,reduction.name="umap.peaks",reduction.key="peaksUMAP_") 

## ATAC harmony integration
clean <- RunHarmony(clean, group.by.vars="individual_ID", reduction="lsi",
                       assay.use="peaks",reduction.save="harmony.peaks", project.dim=F)
clean <- RunUMAP(clean,reduction="harmony.peaks",dims=1:30,reduction.name="harmony.peaks.umap")

In [ ]:
## integration and jointly clustering
#clean <- FindMultiModalNeighbors(clean, reduction.list=list("harmony.rna","harmony.peaks"), dims.list=list(1:30,2:50))
#clean <- RunUMAP(clean,nn.name="weighted.nn", reduction.name="wnn.umap",reduction.key="wnnUMAP_")
#clean <- FindClusters(clean, graph.name = "wsnn", algorithm = 3, verbose = T, resolution = 1)

## integration and jointly clustering
clean <- FindMultiModalNeighbors(clean, reduction.list=list("harmony.rna","harmony.CTpeaks"), dims.list=list(1:30,2:50))
clean <- RunUMAP(clean,nn.name="weighted.nn", reduction.name="wnn.umap",reduction.key="wnnUMAP_")
clean <- FindClusters(clean, graph.name = "wsnn", algorithm = 3, verbose = T, resolution = 1)

In [ ]:
DimPlot(clean,reduction = "wnn.umap",group.by = "cluster_celltype",label = T)
meta <- clean@meta.data
table(meta$wsnn_res.1)

In [ ]:
##remove the cells with different label with majority of the clusters for cleaning again
#filter out cells that are not assigned the same cell type as their neighbors. Additional doublet precaution 
Idents(clean)<-clean$wsnn_res.1
tab<-table(clean$wsnn_res.1, clean$major.cell.type.prediction.mathy)
tmp<-colnames(tab)[apply(tab,1,which.max)]
names(tmp)<-levels(clean)
clean<-RenameIdents(clean,tmp)
clean$cluster_celltype<-Idents(clean)
clean$agree<-ifelse(clean$cluster_celltype==clean$major.cell.type.prediction.mathy, 1,0)

table(clean$agree)

In [ ]:
## subsetting based on the criteria
clean <- subset(clean, subset=agree==1) 
clean

In [ ]:
## redo normalization, integration for visualization
################################################################################
#################### Harmony integration on each omic ##########################
################################################################################
library(harmony)

## Normalization and clutering
DefaultAssay(clean) <- "PC"

clean <- SCTransform(clean,assay="PC",new.assay.name="SCT",vars.to.regress = c("percent.mt"),verbose=T) %>% 
        RunPCA(ndims=30) %>% 
        FindNeighbors(dims = 1:30) %>% 
        RunUMAP(dims = 1:30, reduction.name="umap.rna",reduction.key = "rnaUMAP_")

## RNA harmony integration
clean<-RunHarmony(clean, group.by.vars="individual_ID",reduction="pca",assay.use="SCT", 
                       reduction.save="harmony.rna",theta = 1)
clean<-RunUMAP(clean,reduction="harmony.rna",dims=1:30,reduction.name="harmony.rna.umap")

# finding RNA clusters
clean<-FindNeighbors(clean,dims=1:30, reduction="harmony.rna")
clean<-FindClusters(clean, graph.name="SCT_nn", resolution=1)
clean$RNA_harmony_cluster <- clean$seurat_clusters

## CTpeaks part (no call peaks yet)
DefaultAssay(clean)<-"CTpeaks"

# Normalization and UMAP on ATAC seq 
clean <- RunTFIDF(clean) %>% 
                FindTopFeatures(min.cutoff='q0') %>% 
                RunSVD() %>% 
                RunUMAP(reduction='lsi',dims=2:50,reduction.name="umap.CTpeaks",reduction.key="CTpeaksUMAP_") 

## ATAC harmony integration
clean <- RunHarmony(clean, group.by.vars="individual_ID", reduction="lsi",
                       assay.use="CTpeaks",reduction.save="harmony.CTpeaks", project.dim=F)
clean <- RunUMAP(clean,reduction="harmony.CTpeaks",dims=1:30,reduction.name="harmony.CTpeaks.umap")


## peaks part (no call peaks yet)
DefaultAssay(clean)<-"peaks"

# Normalization and UMAP on ATAC seq 
clean <- RunTFIDF(clean) %>% 
                FindTopFeatures(min.cutoff='q0') %>% 
                RunSVD() %>% 
                RunUMAP(reduction='lsi',dims=2:50,reduction.name="umap.peaks",reduction.key="peaksUMAP_") 

## ATAC harmony integration
clean <- RunHarmony(clean, group.by.vars="individual_ID", reduction="lsi",
                       assay.use="peaks",reduction.save="harmony.peaks", project.dim=F)
clean <- RunUMAP(clean,reduction="harmony.peaks",dims=1:30,reduction.name="harmony.peaks.umap")

In [ ]:
## integration and jointly clustering
#clean <- FindMultiModalNeighbors(clean, reduction.list=list("harmony.rna","harmony.peaks"), dims.list=list(1:30,2:50))
#clean <- RunUMAP(clean,nn.name="weighted.nn", reduction.name="wnn.umap",reduction.key="wnnUMAP_")
#clean <- FindClusters(clean, graph.name = "wsnn", algorithm = 3, verbose = T, resolution = 1)

## integration and jointly clustering
clean <- FindMultiModalNeighbors(clean, reduction.list=list("harmony.rna","harmony.CTpeaks"), dims.list=list(1:30,2:50))
clean <- RunUMAP(clean,nn.name="weighted.nn", reduction.name="wnn.umap",reduction.key="wnnUMAP_")
clean <- FindClusters(clean, graph.name = "wsnn", algorithm = 3, verbose = T, resolution = 1)

In [ ]:
clean <- readRDS("03.clean_object_10.31.rds")

In [ ]:
table(clean$regions)
table(clean$cluster_celltype)
table(clean$regions,clean$cluster_celltype)

In [ ]:
umap_theme <- theme_dr()+theme(panel.grid.major = element_blank(), 
                                            panel.grid.minor = element_blank(),
                                            panel.background = element_blank(), 
                                            axis.line = element_line(colour = "black"))
p1 <- DimPlot(clean, label = T, repel = TRUE, 
              reduction = "wnn.umap",group.by = "cluster_celltype")+umap_theme+ggtitle("wnn.umap")
p2 <- DimPlot(clean, label = T, repel = TRUE, 
              reduction = "harmony.rna.umap",group.by = "cluster_celltype")+umap_theme+ggtitle("harmony.rna")
p3 <- DimPlot(clean, label = T, repel = TRUE, 
              reduction = "harmony.atac.umap",group.by = "cluster_celltype")+umap_theme+ggtitle("harmony.atac")

#ggsave(filename = "./Figures/UMAP/4_clean_object_CT_ALL.pdf",
#    patchwork::wrap_plots(p1,p2,p3, ncol = 3),
#      scale = 1, width = 17, height = 5)
p1
p2
p3

In [ ]:
#######################################################
## region umap ########################################
#######################################################
col1=c('#F06719','#33A65C','#23767C','#E03426','#1BA3C6',"#A26DC2","#FCB905","#EB73B3")
names(col1)=c('Astrocyte','Excitatory','Inhibitory','Microglia','Oligodendrocyte',"OPC","Endothelial","VLMC/Per")
cols=c(unname(col1),'grey90')
names(cols)=c(names(col1),'other')
cols

In [ ]:
umap_theme <- theme_dr()+theme(panel.grid.major = element_blank(), 
                                            panel.grid.minor = element_blank(),
                                            panel.background = element_blank(), 
                                            axis.line = element_line(colour = "black"))

pdf(file='./Figures/UMAP/Regions/BrainRegion_UMAP_coloredbyCelltype.pdf',width = 5,height = 5)
meta <- clean@meta.data
regions <- unique(clean$regions)
for (r in regions){
    print(r)
    tmp1=meta[meta$regions==r,]
    tmp1$mycol=tmp1$major.cell.type.prediction.mathy
    tmp2=meta[meta$regions!=r,]
    tmp2$mycol=rep('other',nrow(tmp2))
    tmp=rbind(tmp1,tmp2)
    tmp=tmp[rownames(meta),]
    clean.tmp <- clean
    clean.tmp@meta.data=tmp
    myorder=names(cols)
print(DimPlot(clean.tmp,reduction = 'wnn.umap',label = F,group.by = 'mycol',order=myorder,cols=cols,pt.size = 0.2)+umap_theme+NoLegend()+ggtitle(paste(r)))
}

print(DimPlot(clean,reduction = 'wnn.umap',label = F,group.by = 'cluster_celltype',cols=cols,pt.size = 0.2)+umap_theme+NoLegend()+ggtitle("ALL"))
#print(DimPlot(clean,reduction = 'wnn.umap',label = F,group.by = 'cluster_celltype',cols=cols,pt.size = 0.2)+umap_theme+ggtitle("ALL"))
print(DimPlot(clean,reduction = 'harmony.rna.umap',label = F,group.by = 'cluster_celltype',cols=cols,pt.size = 0.2)+umap_theme+NoLegend()+ggtitle("RNA"))
print(DimPlot(clean,reduction = 'harmony.atac.umap',label = F,group.by = 'cluster_celltype',cols=cols,pt.size = 0.2)+umap_theme+NoLegend()+ggtitle("ATAC"))
dev.off()

In [ ]:
clean$cluster_celltype <- factor(x = clean$cluster_celltype,levels = sort(levels(clean$cluster_celltype)))
#clean$cluster_celltype <- factor(x = clean$cluster_celltype,levels = names(sort(table(clean$cluster_celltype),decreasing = T)))

Idents(clean) <- clean$cluster_celltype

In [ ]:
saveRDS(clean, file = "03.clean_object.10.31.rds")

In [ ]:
### focus on PFC first
PFC_object <- subset(clean, subset = regions == "PFC")
DefaultAssay(PFC_object) <- "PC"
################################################################################
############################## log normalization ###############################
################################################################################
PFC_object <- NormalizeData(PFC_object, normalization.method = "LogNormalize", scale.factor = 10000)
all.genes <- rownames(PFC_object)
PFC_object <- ScaleData(PFC_object, features = all.genes)

## find variable features 
PFC_object <- FindVariableFeatures(PFC_object, selection.method = "vst", nfeatures = 2000)

PFC_object

saveRDS(PFC_object, file = "cellbender_PFC_object_10.31.rds")

In [ ]:
### focus on PFC first
HIP_object <- subset(clean, subset = regions == "HIP")
DefaultAssay(HIP_object) <- "PC"
################################################################################
############################## log normalization ###############################
################################################################################
HIP_object <- NormalizeData(HIP_object, normalization.method = "LogNormalize", scale.factor = 10000)
all.genes <- rownames(HIP_object)
HIP_object <- ScaleData(HIP_object, features = all.genes)

## find variable features 
HIP_object <- FindVariableFeatures(HIP_object, selection.method = "vst", nfeatures = 2000)

HIP_object

saveRDS(HIP_object, file = "cellbender_HIP_object_10.31.rds")

In [ ]:
### focus on PFC first
EC_object <- subset(clean, subset = regions == "EC")
DefaultAssay(EC_object) <- "PC"
################################################################################
############################## log normalization ###############################
################################################################################
EC_object <- NormalizeData(EC_object, normalization.method = "LogNormalize", scale.factor = 10000)
all.genes <- rownames(EC_object)
EC_object <- ScaleData(EC_object, features = all.genes)

## find variable features 
EC_object <- FindVariableFeatures(EC_object, selection.method = "vst", nfeatures = 2000)

EC_object

saveRDS(EC_object, file = "cellbender_EC_object_10.31.rds")

In [ ]:
####################################################################
##-- preparing for cell type proportional analysis using scCODA --##
####################################################################

##conda install numpy scipy joblib scikit-learn --force-reinstall
## since using seurat 5, need to change some assay to seurat 3/4 for some functions
# backup
outobj <- clean
outobj[["RNA"]] <- as(outobj[["RNA"]], "Assay")
DefaultAssay(outobj) <- "RNA"
outobj@assays$RNA$data <- NULL
outobj
## output as a h5ad format, need to install the following things
sceasy::convertFormat(outobj, from="seurat", to="anndata",
                       outFile='clean_object_for_proportional_analysis.h5ad')

In [ ]:
saveRDS(clean, file = "./Analysis/03.clean_object.rds", ascii = FALSE, version = NULL,
        compress = TRUE, refhook = NULL)

In [ ]:
donor_map <- c(
  "UT04" = "UT04",
  "UT09" = "UT09",
  "UT2105" = "UT2105",
  "NIH01" = "BEB18077",
  "NIH02" = "BEB18077",
  "NIH03" = "BEB18077",
  "NIH04" = "BEB18110",
  "NIH05" = "BEB18110",
  "NIH06" = "BEB18110",
  "NIH10" = "BEB19074",
  "NIH11" = "BEB19074",
  "NIH12" = "BEB19074",
  "NIH13" = "BEB19080",
  "NIH14" = "BEB19080",
  "NIH15" = "BEB19080",
  "NIH16" = "BEB20005",
  "NIH17" = "BEB20005",
  "NIH18" = "BEB20005",
  "NIH28" = "HCTZD",
  "NIH29" = "HCTZD",
  "NIH30" = "HCTZD"
)
meta = clean@meta.data
# Apply the mapping to create a new metadata column 'donor_id'
meta$donor_id <- donor_map[meta$individual_ID]
table(meta$donor_id)
meta$new_label = paste(meta$donor_id, meta$individual_ID, sep = "_")
table(meta$new_label)

In [ ]:
clean@meta.data = meta

In [ ]:
colnames(clean@meta.data)
# meta <- clean@meta.data

### snRNA-seq QC plots
# p1<- ggplot(meta, aes(x=cluster_celltype, y=nFeature_RNA, fill=diagnosis)) + 
#     geom_boxplot()+theme_classic()+ggtitle("PFC_major_celltype_nFeatures_by_diagnosis")
# p2<- ggplot(meta, aes(x=cluster_celltype, y=nFeature_RNA, fill=individual_ID)) + 
#     geom_boxplot()+theme_classic()+ggtitle("PFC_major_celltype_nFeatures_by_sample")

    ### snRNA-seq QC plots
#region_colors <- c("PFC" = "#c25757ff", "EC" = "#825ca6ff", "HIP" = "#3f78c199")
p1 <- ggplot(meta, aes(x=new_label, y=log(nCount_RNA,base =10), fill=regions)) + 
    geom_boxplot()+theme_classic()+ggtitle("nCount_RNA_by_sample") +
    scale_fill_manual(values=c("#825ca6ff", "#3f78c199","#c25757ff")) +
    theme(legend.position = "none",axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

p2 <- ggplot(meta, aes(x=new_label, y=nFeature_RNA,base =10, fill=regions)) + 
    geom_boxplot()+theme_classic()+ggtitle("nFeature_RNA_by_sample") +
    scale_fill_manual(values=c("#825ca6ff", "#3f78c199","#c25757ff")) +
    theme(legend.position = "none",axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

p3 <- ggplot(meta, aes(x=new_label, y=log(nCount_ATAC,base =10), fill=regions)) + 
    geom_boxplot()+theme_classic()+ggtitle("nCount_ATAC_by_sample") +
    scale_fill_manual(values=c("#825ca6ff", "#3f78c199","#c25757ff")) +
    theme(legend.position = "none",axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

p4 <- ggplot(meta, aes(x=new_label, y=TSS.enrichment, fill=regions)) + 
    geom_boxplot()+theme_classic()+ggtitle("TSS_enrichment_by_sample") +
    scale_fill_manual(values=c("#825ca6ff", "#3f78c199","#c25757ff")) +
    theme(legend.position = "none",axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

    ## fragments plot
p5 <- FragmentHistogram(object = clean, group.by = 'new_label')

In [ ]:
# Ensure these metadata columns exist: sample_id and cell_type
# Adjust column names if needed
meta <- clean@meta.data %>%
  dplyr::select(new_label, cluster_celltype, nFeature_RNA)

# Compute average number of genes per cell for each sample × cell type
avg_gene_counts <- meta %>%
  group_by(new_label, cluster_celltype) %>%
  summarise(avg_nFeature = mean(nFeature_RNA), .groups = "drop")



In [ ]:
# Convert to wide matrix for heatmap (rows = cell types, columns = samples)
heatmap_matrix <- avg_gene_counts %>%
  tidyr::pivot_wider(names_from = new_label, values_from = avg_nFeature, values_fill = 0) %>%
  column_to_rownames("cluster_celltype") %>%
  as.matrix()

# Plot heatmap
library(ComplexHeatmap)

# Create the heatmap
p6 = Heatmap(
      heatmap_matrix,
      name = "# Genes",
      cluster_rows = FALSE,
      cluster_columns = FALSE,
      cell_fun = function(j, i, x, y, width, height, fill) {
        grid.text(
          label = round(heatmap_matrix[i, j], 1),
          x = x, y = y,
          rot = 90,  # Rotate the text vertically
          gp = gpar(fontsize = 10, col = "black")
        )
      }
    )
# Save the heatmap plot
pdf("./Figures/all_object_QC_plot_2.pdf",width = 12,height = 6)
p6
dev.off()

In [ ]:
design = "AABBCCCC
          AABBCCCC
          DDEECCCC
          DDEE...."

pdf("./Figures/all_object_QC_plot.pdf",width = 20,height = 12)
patchwork::wrap_plots(A = p1,B= p3,C = p5, D = p2,E = p4,design = design)
dev.off()